# Converter.ipynb — Exportador de Notebooks a PDF (Google Colab)

Este notebook permite **convertir cualquier archivo `.ipynb`** en un **PDF con las gráficas de Plotly en alta resolución (PNG HD)** directamente desde Google Colab. Se ocultan celdas técnicas de la conversión de manera automática.

---

### **INSTRUCCIONES**

#### 1️⃣ **Abrir Colab**
Abre o sube este archivo `Converter.ipynb` en Google Colab.

#### 2️⃣ **Carga tus archivos adicionales**

Si tu notebook utiliza archivos como CSV, Excel, imágenes u otros recursos que se cargan desde el disco, debes subirlos también a este mismo entorno de Colab antes de ejecutar el converter.

#### 3️⃣ **Ejecutar**

Da click en el ícono para correr el código.


#### 4️⃣ **Cargar tu notebook**

Después de ejecutar el código, el programa te pedirá subir tu `.ipynb`.  

---

In [6]:
# ================================================================
# IPYNB → PDF (Colab) — Sirve incluso para gráficas Plotly
# ================================================================

# 0) DEPENDENCIAS (se instalan solo si faltan)
def _pip_install_if_needed():
    import importlib, sys, subprocess
    pkgs = [
        ("plotly", "plotly==5.24.1"),
        ("kaleido", "kaleido==0.2.1"),
        ("nbconvert", "nbconvert==7.*"),
        ("nbformat", "nbformat"),
        ("pyppeteer", "pyppeteer==2.0.0"),
        ("pyee", "pyee==11.1.0"),
        ("nest_asyncio", "nest_asyncio"),
    ]
    for mod, spec in pkgs:
        try:
            importlib.import_module(mod)
        except Exception:
            print(f"[INFO] Installing {spec} …")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", spec])

_pip_install_if_needed()

# 1) PARÁMETROS DEL USUARIO (ajusta aquí)
NOTEBOOK_PATH = None           # Ruta al .ipynb en /content/...  (o deja None para subirlo)
OUTPUT_PDF   = None            # None => <nombre>_export.pdf
HIDE_CODE    = False           # True = ocultar TODAS las celdas de código (si lo deseas)
ALLOW_ERRORS = True            # True = continuar si alguna celda falla
PAGE_FORMAT  = "A4"            # "A4" o "Letter"
IMG_WIDTH    = 1200            # px
IMG_HEIGHT   = 800             # px
IMG_SCALE    = 2               # 2-3 recomendado

# 2) LIBRERÍAS
import os, asyncio, tempfile, nbformat, pathlib
from nbconvert.preprocessors import ExecutePreprocessor
from nbconvert import HTMLExporter
from traitlets.config import Config
import nest_asyncio; nest_asyncio.apply()

# 3) CARGAR ARCHIVO (si NO diste ruta, abrimos selector)
if NOTEBOOK_PATH is None:
    try:
        from google.colab import files
        print("🔼 Selecciona tu archivo .ipynb …")
        up = files.upload()
        NOTEBOOK_PATH = list(up.keys())[0]
    except Exception as e:
        raise RuntimeError("En Colab puedes subir el .ipynb con files.upload() o fija NOTEBOOK_PATH.") from e

# 4) CONFIGURAR NOMBRE DE SALIDA
if OUTPUT_PDF is None:
    base = os.path.splitext(os.path.basename(NOTEBOOK_PATH))[0]
    OUTPUT_PDF = f"{base}_export.pdf"

# 5) CELDA DE SETUP PARA PLOTLY→PNG HD (se inyecta, con TAG 'hide_input')
_PLOTLY_SETUP_CODE = f"""
import plotly.io as pio
if getattr(pio, "kaleido", None) is None:
    raise RuntimeError("Kaleido no disponible; usa plotly==5.24.1 y kaleido==0.2.1.")
pio.kaleido.scope.default_format = "png"
pio.kaleido.scope.default_width  = {IMG_WIDTH}
pio.kaleido.scope.default_height = {IMG_HEIGHT}
pio.kaleido.scope.default_scale  = {IMG_SCALE}
pio.renderers.default = "png"
"""

def _prepend_setup_cell(nb):
    # 🚩 metadatos con tag 'hide_input' para ocultar SOLO esta celda en el PDF
    cell = nbformat.v4.new_code_cell(
        _PLOTLY_SETUP_CODE,
        metadata={"tags": ["hide_input"]}
    )
    first_src = nb.cells[0].source if nb.cells else ""
    if _PLOTLY_SETUP_CODE.strip() not in first_src:
        nb.cells.insert(0, cell)
    return nb

# 6) ASEGURAR CHROMIUM PARA PYPPETEER (sincrónico)
def _ensure_chromium():
    from pyppeteer.chromium_downloader import check_chromium, download_chromium, chromium_executable
    if not check_chromium():
        print("[INFO] Descargando Chromium (una sola vez)…")
        download_chromium()  # sin await en pyppeteer 2.0.0
    return chromium_executable()

# 7) HTML → PDF con Pyppeteer
async def _html_to_pdf_with_pyppeteer(html_path, pdf_path, page_format="A4"):
    from pyppeteer import launch
    exec_path = _ensure_chromium()
    browser = await launch(executablePath=exec_path, args=["--no-sandbox", "--disable-dev-shm-usage"])
    page = await browser.newPage()
    file_url = pathlib.Path(html_path).absolute().as_uri()
    await page.goto(file_url, {"waitUntil": "networkidle0"})

    if hasattr(page, "emulateMediaType"):
        await page.emulateMediaType("print")
    elif hasattr(page, "emulateMedia"):
        await page.emulateMedia("print")

    await page.waitForSelector("body")
    await page.pdf(
        path=pdf_path,
        printBackground=True,
        format=page_format,
        margin={"top": "12mm", "bottom": "12mm", "left": "12mm", "right": "12mm"},
    )
    await browser.close()

# 8) PIPELINE: Ejecutar .ipynb → HTML → PDF
def export_ipynb_to_pdf(ipynb_path, out_pdf_path, hide_code=False, allow_cell_errors=True, page_format="A4"):
    assert os.path.exists(ipynb_path), f"No se encontró el archivo: {ipynb_path}"

    # Leer notebook e inyectar setup de Plotly (tag 'hide_input')
    with open(ipynb_path, "r", encoding="utf-8") as f:
        nb = nbformat.read(f, as_version=4)
    nb = _prepend_setup_cell(nb)

    # Ejecutar notebook (captura PNGs de Plotly)
    workdir = os.path.dirname(ipynb_path) or "."
    ep = ExecutePreprocessor(timeout=1200, kernel_name="python3", allow_errors=bool(allow_cell_errors))
    ep.preprocess(nb, {"metadata": {"path": workdir}})

    # Exportar a HTML:
    #  - Si HIDE_CODE=True → se ocultan TODAS las celdas de código (opcional).
    #  - Si HIDE_CODE=False → se muestran todas, EXCEPTO las que tengan tag 'hide_input'.
    c = Config()
    c.HTMLExporter.exclude_input_prompt  = True
    c.HTMLExporter.exclude_output_prompt = True

    if hide_code:
        c.TemplateExporter.exclude_input = True
    else:
        # Ocultar solo celdas con tag 'hide_input'
        c.TemplateExporter.exclude_input = False
        c.TagRemovePreprocessor.remove_input_tags = {"hide_input"}
        c.HTMLExporter.preprocessors = ["nbconvert.preprocessors.TagRemovePreprocessor"]

    html_exporter = HTMLExporter(config=c)
    body, _ = html_exporter.from_notebook_node(nb)

    # Guardar HTML temporal
    tmp_html = os.path.join(tempfile.gettempdir(), "notebook_export.html")
    with open(tmp_html, "w", encoding="utf-8") as f:
        f.write(body)

    # HTML → PDF
    loop = asyncio.get_event_loop()
    loop.run_until_complete(_html_to_pdf_with_pyppeteer(tmp_html, out_pdf_path, page_format=page_format))

    print(f"✅ PDF generado: {out_pdf_path}")
    return out_pdf_path

# 9) EJECUTAR
print("📄 Notebook:", NOTEBOOK_PATH)
print("🖨️ Exportando a:", OUTPUT_PDF)
export_ipynb_to_pdf(
    NOTEBOOK_PATH,
    OUTPUT_PDF,
    hide_code=HIDE_CODE,          # ← deja False para ver todo el código excepto la celda técnica
    allow_cell_errors=ALLOW_ERRORS,
    page_format=PAGE_FORMAT,
)

# 10) DESCARGA (si estás en Colab)
try:
    from google.colab import files
    files.download(OUTPUT_PDF)
except Exception:
    pass

print("🎉 Listo.")


🔼 Selecciona tu archivo .ipynb …


Saving Reto.ipynb to Reto.ipynb
📄 Notebook: Reto.ipynb
🖨️ Exportando a: Reto_export.pdf
✅ PDF generado: Reto_export.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎉 Listo.


---

### <small>**Tiempos de ejecución y errores**

<small> El proceso puede tardar entre 2 y 5 minutos, dependiendo del tamaño de tu notebook, la cantidad de gráficas o imágenes, y la velocidad de Colab. Durante este tiempo, es normal que no aparezca ningún mensaje nuevo. Solo espera a que aparezca la línea:

<small>`✅ PDF generado: /content/Nombre_export.pdf`

<small> Si el mensaje termina con Traceback, ImportError, o FileNotFoundError, significa que hubo un problema real (por ejemplo, faltan archivos o rutas incorrectas). Si el proceso lleva más de 7 minutos sin mostrar progreso, detén la celda y vuelve a ejecutarla.

<small>Los errores más comunes son:

* <small>Falta un archivo CSV o Excel en /content/.

* <small>El notebook tiene celdas interactivas (por ejemplo, input() o plotly.show() en modo web).

* <small>Conexión lenta de Colab.

---
<small>El script instala automáticamente las dependencias necesarias:

- <small>**Plotly 5.24.1** + **Kaleido 0.2.1** → exporta las gráficas como imágenes PNG.  
- **nbconvert 7.x** + **pyppeteer 2.0.0** → convierte el HTML generado a PDF.  
- **nest_asyncio** → permite que Colab ejecute procesos asincrónicos sin errores.

---

## Créditos
Desarrollado por **Abi Rodríguez**.
Este notebook fue diseñado como una herramienta personal para convertir archivos de Google Colab `.ipynb` a PDF con gráficas de Plotly en alta resolución.

Con apoyo de **ChatGPT (OpenAI)** para depurar errores técnicos.